<a href="https://colab.research.google.com/github/Iktaik-Kim/MCP/blob/main/%EC%9A%B4%ED%95%AD_%EB%AA%A8%EB%8B%88%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import os

print("=== 부산항 정시 도착 모니터링 시스템 (수정버전) ===")

# 파일 업로드
uploaded = files.upload()
uploaded_files = list(uploaded.keys())
if uploaded_files:
    file_name = uploaded_files[0]
    print(f"✓ 업로드된 파일: {file_name}")
else:
    print("❌ 파일이 업로드되지 않습니다.")
    exit()

# 엑셀 파일 읽기
try:
    df = pd.read_excel(file_name)
    print(f"✓ 파일 읽기 성공: {len(df)}개의 선박 스케줄")
except Exception as e:
    print(f"❌ 파일 읽기 실패: {e}")
    exit()

class VesselMonitoringSystem:
    """수정된 선박 모니터링 시스템"""

    def __init__(self):
        self.busan_port = (35.1, 129.04)  # 부산항 좌표
        self.current_date = datetime(2025, 10, 8)  # 현재 날짜 고정 (실제로는 datetime.now())

    def get_vessel_status(self, vessel_name, departure_date, arrival_date):
        """선박 상태 조회 (출발 전/운항 중/도착 완료)"""
        departure_date = pd.to_datetime(departure_date)
        arrival_date = pd.to_datetime(arrival_date)

        # 선박 상태 판단
        if self.current_date < departure_date:
            status = 'BEFORE_DEPARTURE'
            progress = 0
        elif self.current_date > arrival_date:
            status = 'ARRIVED'
            progress = 100
        else:
            status = 'UNDERWAY'
            # 진행률 계산
            total_days = (arrival_date - departure_date).days
            days_passed = (self.current_date - departure_date).days
            progress = min(max(days_passed / total_days, 0), 1) if total_days > 0 else 0.5

        return status, round(progress * 100, 1)

    def get_simulated_position(self, vessel_name, departure_date, arrival_date, status, progress):
        """선박 위치 시뮬레이션"""
        start_lat, start_lon = 33.7, -118.2  # LA 항 좌표
        end_lat, end_lon = self.busan_port

        if status == 'BEFORE_DEPARTURE':
            # 출발 전: LA 항구에 대기 중
            current_lat, current_lon = start_lat, start_lon
            speed = 0
            course = 0
        elif status == 'ARRIVED':
            # 도착 완료: 부산항 정박 중
            current_lat, current_lon = end_lat, end_lon
            speed = 0
            course = 0
        else:
            # 운항 중: 진행률에 따른 위치
            progress_ratio = progress / 100.0
            current_lat = start_lat + (end_lat - start_lat) * progress_ratio
            current_lon = start_lon + (end_lon - start_lon) * progress_ratio
            speed = np.random.uniform(18, 22)
            course = 280

        return {
            'vessel': vessel_name,
            'timestamp': self.current_date,
            'latitude': round(current_lat, 4),
            'longitude': round(current_lon, 4),
            'speed': round(speed, 1),
            'course': course,
            'status': status,
            'progress_percent': progress
        }

    def calculate_eta(self, vessel_data, current_position, status, progress):
        """도착 예정 시간 계산"""
        if status == 'BEFORE_DEPARTURE':
            # 출발 전인 경우
            scheduled_departure = pd.to_datetime(vessel_data['LA항 출항일'])
            scheduled_arrival = pd.to_datetime(vessel_data['부산항 도착일'])

            return {
                'remaining_distance_nm': 5500,
                'estimated_hours': (scheduled_arrival - scheduled_departure).days * 24,
                'calculated_eta': scheduled_arrival,
                'on_schedule_status': 'SCHEDULED',
                'on_schedule_status_kor': '예정대로',
                'hours_difference': 0,
                'notes': '출발 대기 중'
            }
        elif status == 'ARRIVED':
            # 이미 도착한 경우
            actual_arrival = pd.to_datetime(vessel_data['부산항 도착일'])
            return {
                'remaining_distance_nm': 0,
                'estimated_hours': 0,
                'calculated_eta': actual_arrival,
                'on_schedule_status': 'ARRIVED',
                'on_schedule_status_kor': '도착 완료',
                'hours_difference': 0,
                'notes': '도착 완료'
            }
        else:
            # 운항 중인 경우
            current_lat = current_position['latitude']
            current_lon = current_position['longitude']
            current_speed = current_position['speed']

            # 부산항까지 남은 거리 계산
            remaining_distance = self._calculate_remaining_distance(
                current_lat, current_lon, progress
            )

            # 예상 소요 시간 계산
            hours_to_arrival = remaining_distance / current_speed if current_speed > 0 else 999
            calculated_eta = self.current_date + timedelta(hours=hours_to_arrival)

            # 정시 도착 여부 판단
            scheduled_arrival = pd.to_datetime(vessel_data['부산항 도착일'])
            time_diff = calculated_eta - scheduled_arrival
            hours_diff = time_diff.total_seconds() / 3600

            if abs(hours_diff) <= 6:
                status_code = 'ON_TIME'
                status_kor = '정시도착 예상'
            elif hours_diff > 6:
                status_code = 'DELAYED'
                status_kor = f'지연 예상 (+{int(hours_diff-6)}시간)'
            else:
                status_code = 'EARLY'
                status_kor = f'조기 도착 예상 ({int(abs(hours_diff)-6)}시간)'

            return {
                'remaining_distance_nm': round(remaining_distance, 1),
                'estimated_hours': round(hours_to_arrival, 1),
                'calculated_eta': calculated_eta,
                'on_schedule_status': status_code,
                'on_schedule_status_kor': status_kor,
                'hours_difference': round(hours_diff, 1),
                'notes': '운항 중'
            }

    def _calculate_remaining_distance(self, lat, lon, progress):
        """남은 거리 계산"""
        total_distance = 5500  # LA-부산 간 총 거리 (해리)
        remaining = total_distance * (1 - progress / 100.0)
        return max(0, remaining)

def monitor_vessel_arrival_performance(df):
    """선박 도착 성능 모니터링"""

    vms = VesselMonitoringSystem()
    monitoring_results = []

    print("\n=== 선박 위치 모니터링 시작 ===")
    print(f"현재 기준 시간: {vms.current_date}")

    for idx, vessel in df.iterrows():
        print(f"모니터링 중: {vessel['선사']} - {vessel['선박명']}")

        # 선박 상태 확인
        status, progress = vms.get_vessel_status(
            vessel['선박명'],
            vessel['LA항 출항일'],
            vessel['부산항 도착일']
        )

        # 선박 위치 조회
        position = vms.get_simulated_position(
            vessel['선박명'],
            vessel['LA항 출항일'],
            vessel['부산항 도착일'],
            status,
            progress
        )

        # ETA 계산
        eta_analysis = vms.calculate_eta(vessel, position, status, progress)

        # 결과 저장
        result = {
            '선사': vessel['선사'],
            '선박명': vessel['선박명'],
            '항차': vessel['항차'],
            'LA_출항일': vessel['LA항 출항일'],
            '부산_도착예정일': vessel['부산항 도착일'],
            '선박_상태': status,
            '선박_상태_한글': {
                'BEFORE_DEPARTURE': '출발 대기',
                'UNDERWAY': '운항 중',
                'ARRIVED': '도착 완료'
            }[status],
            '운항_진행률': progress,
            '현재_위도': position['latitude'],
            '현재_경도': position['longitude'],
            '현재_속도': position['speed'],
            '부산항_남은거리': eta_analysis['remaining_distance_nm'],
            '예상_소요시간': eta_analysis.get('estimated_hours', 0),
            '계산된_도착시간': eta_analysis['calculated_eta'],
            '정시도착_상태': eta_analysis['on_schedule_status'],
            '정시도착_상태_한글': eta_analysis['on_schedule_status_kor'],
            '도착시간_차이': eta_analysis.get('hours_difference', 0),
            '비고': eta_analysis.get('notes', ''),
            '모니터링_시간': vms.current_date
        }
        monitoring_results.append(result)

    return pd.DataFrame(monitoring_results)

def generate_performance_report(monitoring_df):
    """성능 리포트 생성"""

    print("\n" + "="*50)
    print("부산항 정시 도착 모니터링 리포트")
    print("="*50)

    # 선박 상태별 통계
    status_counts = monitoring_df['선박_상태_한글'].value_counts()
    print("\n📊 선박 상태 현황:")
    for status, count in status_counts.items():
        print(f"   {status}: {count}척")

    # 운항 중인 선박만 필터링하여 정시 도착률 계산
    underway_vessels = monitoring_df[monitoring_df['선박_상태'] == 'UNDERWAY']

    if len(underway_vessels) > 0:
        on_time_underway = len(underway_vessels[underway_vessels['정시도착_상태'] == 'ON_TIME'])
        on_time_rate_underway = (on_time_underway / len(underway_vessels)) * 100

        print(f"\n✅ 운항 중인 선박 정시 도착률: {on_time_rate_underway:.1f}% ({on_time_underway}/{len(underway_vessels)}척)")

        # 지연 예상 선박
        delayed_vessels = underway_vessels[underway_vessels['정시도착_상태'] == 'DELAYED']
        if len(delayed_vessels) > 0:
            print(f"🚨 지연 예상 선박:")
            for idx, vessel in delayed_vessels.iterrows():
                print(f"   - {vessel['선사']} {vessel['선박명']}: {vessel['정시도착_상태_한글']}")

    # 출발 대기 중인 선박
    waiting_vessels = monitoring_df[monitoring_df['선박_상태'] == 'BEFORE_DEPARTURE']
    if len(waiting_vessels) > 0:
        print(f"\n⏰ 출발 대기 중인 선박: {len(waiting_vessels)}척")
        for idx, vessel in waiting_vessels.head(3).iterrows():  # 상위 3개만 출력
            days_until_departure = (pd.to_datetime(vessel['LA_출항일']) - pd.to_datetime(vessel['모니터링_시간'])).days
            print(f"   - {vessel['선사']} {vessel['선박명']}: {days_until_departure}일 후 출발 예정")

# 메인 실행
print("\n모니터링 시스템 시작...")

# 모니터링 실행
monitoring_results = monitor_vessel_arrival_performance(df)

# 리포트 생성
generate_performance_report(monitoring_results)

# 결과 저장
output_file = 'busan_arrival_monitoring_corrected.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # 상세 모니터링 데이터
    monitoring_results.to_excel(writer, sheet_name='실시간_모니터링', index=False)

    # 선사별 요약
    company_summary = monitoring_results.groupby('선사').agg({
        '선박명': 'count',
        '운항_진행률': 'mean',
        '정시도착_상태': lambda x: (x == 'ON_TIME').sum() / len(x) * 100 if len(x) > 0 else 0
    }).round(1)
    company_summary.columns = ['총_선박수', '평균_진행률', '정시도착률(%)']
    company_summary.to_excel(writer, sheet_name='선사별_요약')

    # 상태별 분류
    status_summary = monitoring_results.groupby(['선사', '선박_상태_한글']).size().unstack(fill_value=0)
    status_summary.to_excel(writer, sheet_name='상태별_분류')

print(f"\n💾 수정된 모니터링 결과가 '{output_file}'에 저장되었습니다.")

# 파일 다운로드
files.download(output_file)

print("\n✅ 모니터링 완료! 다운로드된 엑셀 파일을 확인해주세요.")

# 결과 미리보기
print("\n📋 모니터링 결과 미리보기:")
preview_cols = ['선사', '선박명', '선박_상태_한글', '운항_진행률', '정시도착_상태_한글', '계산된_도착시간']
print(monitoring_results[preview_cols].head(10))

=== 부산항 정시 도착 모니터링 시스템 (수정버전) ===


Saving LA_Busan_Schedule_2025-10-20_to_2025-11-10.xlsx.xlsx to LA_Busan_Schedule_2025-10-20_to_2025-11-10.xlsx (1).xlsx
✓ 업로드된 파일: LA_Busan_Schedule_2025-10-20_to_2025-11-10.xlsx (1).xlsx
✓ 파일 읽기 성공: 16개의 선박 스케줄

모니터링 시스템 시작...

=== 선박 위치 모니터링 시작 ===
현재 기준 시간: 2025-10-08 00:00:00
모니터링 중: ONE - ONE VESSEL 100
모니터링 중: EVERGREEN - EVERGREEN VESSEL 101
모니터링 중: YANG MING - YANG MING VESSEL 102
모니터링 중: CMA CGM - CMA CGM VESSEL 103
모니터링 중: MAERSK - MAERSK VESSEL 105
모니터링 중: HMM - HMM VESSEL 104
모니터링 중: MSC - MSC VESSEL 106
모니터링 중: COSCO - COSCO VESSEL 107
모니터링 중: ONE - ONE VESSEL 108
모니터링 중: EVERGREEN - EVERGREEN VESSEL 109
모니터링 중: CMA CGM - CMA CGM VESSEL 111
모니터링 중: YANG MING - YANG MING VESSEL 110
모니터링 중: HMM - HMM VESSEL 112
모니터링 중: MAERSK - MAERSK VESSEL 113
모니터링 중: MSC - MSC VESSEL 114
모니터링 중: COSCO - COSCO VESSEL 115

부산항 정시 도착 모니터링 리포트

📊 선박 상태 현황:
   출발 대기: 16척

⏰ 출발 대기 중인 선박: 16척
   - ONE ONE VESSEL 100: 12일 후 출발 예정
   - EVERGREEN EVERGREEN VESSEL 101: 13일 후 출발 예정
   - YANG MING YAN

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 모니터링 완료! 다운로드된 엑셀 파일을 확인해주세요.

📋 모니터링 결과 미리보기:
          선사                   선박명 선박_상태_한글  운항_진행률 정시도착_상태_한글   계산된_도착시간
0        ONE        ONE VESSEL 100    출발 대기       0       예정대로 2025-11-03
1  EVERGREEN  EVERGREEN VESSEL 101    출발 대기       0       예정대로 2025-11-05
2  YANG MING  YANG MING VESSEL 102    출발 대기       0       예정대로 2025-11-05
3    CMA CGM    CMA CGM VESSEL 103    출발 대기       0       예정대로 2025-11-07
4     MAERSK     MAERSK VESSEL 105    출발 대기       0       예정대로 2025-11-09
5        HMM        HMM VESSEL 104    출발 대기       0       예정대로 2025-11-10
6        MSC        MSC VESSEL 106    출발 대기       0       예정대로 2025-11-12
7      COSCO      COSCO VESSEL 107    출발 대기       0       예정대로 2025-11-14
8        ONE        ONE VESSEL 108    출발 대기       0       예정대로 2025-11-14
9  EVERGREEN  EVERGREEN VESSEL 109    출발 대기       0       예정대로 2025-11-16
